In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import powerlaw
from scipy.stats import uniform
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [2]:
# Scale Factor 
DAY=3600 * 24
G = 6.6743e-11  # m3 kg-1 s-2

In [3]:
# Set number of datapoints (Elisa have 30000 data points)
number=60000000

In [4]:
class SamplingProbability():
    def __init__(self, pdf, domain, isLog=False, **kwargs):
        self.pdf = pdf
        self.domain = domain
        self.isLog = isLog
        self.pdf_args = kwargs
        self._compute_cdf()
    def eval_pdf(self, x):
#         if self.isLog:
#             return np.log10(self.pdf( x, **self.pdf_args ))
        return self.pdf( x, **self.pdf_args )        
    def _compute_cdf(self):
        self.generate_grid_points()
        p = self.eval_pdf(self.grids)
        if self.isLog:
            ln10 = np.log(10) # this normaliiation constant can drop. __normalize will cancel it
            self.cdf = ln10 * np.cumsum( p*self.grids  )
        else:
            self.cdf = np.cumsum(p) # if it is log grid, should not use this formula
        self.__normalize_cdf()
    def generate_grid_points(self, precision=100):
        if self.isLog:
            __lower = np.log10(self.domain[0])
            __upper = np.log10(self.domain[1])
            self.grids = 10**np.linspace(__lower, __upper, precision)
        else:
            self.grids = np.linspace( self.domain[0], self.domain[1], precision )
    def __normalize_cdf(self):
        self.cdf -= np.min(self.cdf)
        self.cdf /= np.max(self.cdf)
        # check cdf normalization
        assert np.max( self.cdf ) == 1 and np.min(self.cdf) == 0
    def generate_samples(self, n_samples):
        # for the explaination, see the wiki page of "inverse sampling transform"
        u = np.random.random(n_samples)
        # numerical inverse of cdf (monotonic) is just reversed grid points
        if self.isLog:
            lgu = np.log10(u)
            return 10**(np.interp( lgu, np.log10(self.cdf), np.log10(self.grids) ))
        else:
            return np.interp( u, self.cdf, self.grids )

In [5]:
def M1powerlaw(x, idx=-2.3, start=10,end=150):
    return x**idx

# define a gaussian funtion with parameters A,X_mean, sigma
def gaus(X,A,X_mean,sigma):
    return A*np.exp(-(X-X_mean)**2/(2*sigma**2))

# define a powerlaw distribution with parameters A, a, cut, offset 
def power(X,A,a,cut,offset):
    flag=(X<cut).astype(int)
    return flag*A*X**a+offset

def dist(X):
    return gaus(X,0.11239582,0.61297987,0.10741941)+power(X,0.03060204,0.45446939,0.5,0.00395605)

In [12]:
# make a power law distribution, with power law index = -2.3
PM1=SamplingProbability(M1powerlaw, [10,150] ) 

# Distribution of metallicity 
PZ=SamplingProbability(dist, [0,1] )

In [13]:
# sample data points from M1 
ListM1 = PM1.generate_samples(number) # in Msun 
ListtM1=ListM1.tolist() 

# Generate mass ratio q:  a power law distribution, with power law index = -0.1 
Listq=powerlaw.rvs(a=0.9,loc=0.1,scale=0.9,size=number,random_state=None)
Listtq=Listq.tolist()

# Generate period: make a power law distribution, with power law index = -0.55
ListP1=powerlaw.rvs(a=0.45,loc=0.15,scale=5.35,size=number,random_state=None) # In Log(P/days) 
ListP=(10**ListP1)*DAY # In second
ListtP=ListP.tolist()

#Generate eccentricity: make a power law distribution, with power law index = -0.42
Liste=powerlaw.rvs(a=0.58,loc=0,scale=1,size=number,random_state=None)
Listte=Liste.tolist()

# Calculate M2 and semi major axis : 
ListM2=ListM1*Listq # in Msun
Listp=(((ListP/(2*np.pi))**2 * G * (ListM1*2e30) * (Listq+1) * (1-Liste**2)**3)**(1/3))/(696340* 10**3) # Semi letus rectum in Rsun 
Listsemi=Listp/(1-Liste**2) # in Rsun 
ListtM2=ListM2.tolist() 
Listtsemi=Listsemi.tolist()



#Create three types of spin list 
ListZeroSpin=np.zeros(number)
ListMaximalSpin=np.ones(number)
ListUniformSpin=uniform.rvs(loc=0,scale=1,size=number,random_state=None)
ListtZeroSpin=ListZeroSpin.tolist()
ListtMaximalSpin=ListMaximalSpin.tolist()
ListtUniformSpin=ListUniformSpin.tolist()


#List of Metallicity 
ListZ1=PZ.generate_samples(number)
ListZ2=ListZ1*1.339912723097303-0.8755311276207431
ListZ=0.02*10**ListZ2
ListtZ=ListZ.tolist()


# Other list in str type 

# Supernova type: choose from rapid, delayed, rapid_gauNS, delayed_gauNS, compact, directcollapse, deathmatrix 
Listsn=['rapid'] * number 
# Choose initial stellar age: choose from zams,tams, shb, cheb, tcheb, sheb
Listtstart=['zams'] * number 
# Choose end time of the simulation: choose from end or broken or specific time 
Listend=['end'] * number 
# Time shcedule for output: choose from List, Interval, all, end, events, eventsrlo or input specific time 
Listtout=['end'] * number 

In [14]:
#Remember to choose a spin distribution here 
dataframe1=pd.DataFrame({'M1': ListtM1,'Z1':ListtZ,'Omega1':ListtUniformSpin,"sn1":Listsn,'tstart1':Listtstart,'M2':ListtM2,'Z2':ListtZ,'Omega2':ListtUniformSpin,'sn2':Listsn,'tstart2':Listtstart,'a':Listtsemi,'e':Listte,'tend':Listend,'dtout':Listtout})

In [15]:
dataframe=dataframe1[dataframe1['M2']>=10]

In [16]:
dataframe

,M1,Z1,Omega1,sn1,tstart1,M2,Z2,Omega2,sn2,tstart2,a,e,tend,dtout
1,45.587101,0.018602,0.819946,rapid,zams,30.293992,0.018602,0.819946,rapid,zams,159.119107,0.181700,end,end
2,12.419267,0.014582,0.497478,rapid,zams,10.399723,0.014582,0.497478,rapid,zams,75.004098,0.031820,end,end
4,32.115085,0.013458,0.781845,rapid,zams,13.855399,0.013458,0.781845,rapid,zams,12206.704762,0.974420,end,end
5,43.821138,0.013625,0.088039,rapid,zams,13.828167,0.013625,0.088039,rapid,zams,28238.398899,0.029301,end,end
6,20.596816,0.046678,0.103425,rapid,zams,19.748795,0.046678,0.103425,rapid,zams,53.133690,0.977234,end,end
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999986,56.945132,0.014939,0.089286,rapid,zams,38.133671,0.014939,0.089286,rapid,zams,29.029981,0.394710,end,end
59999994,54.172572,0.026755,0.112541,rapid,zams,10.316117,0.026755,0.112541,rapid,zams,5036.192354,0.083484,end,end
59999995,11.218546,0.023971,0.761517,rapid,zams,10.892521,0.023971,0.761517,rapid,zams,37.012438,0.069388,end,end
59999996,110.816692,0.011504,0.830626,rapid,zams,87.367434,0.011504,0.830626,rapid,zams,433.957087,0.991082,end,end


In [17]:
dataframe.to_csv('listBin_v15.dat',sep=' ',header=None,index=None)